# Loan Default Prediction: Exploratory Data Analysis

##### Loading the dataset

In [1]:
#Importing pandas
import pandas as pd

In [2]:
df = pd.read_csv('../raw_data/accepted_2007_to_2018Q4.csv', nrows=15000)


/tmp/ipykernel_4105/293979653.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/accepted_2007_to_2018Q4.csv', nrows=15000)


In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


##  Feature Selection: Identifying Key Variables

For this initial analysis, I've selected 11 key columns from the dataset that are most relevant for predicting loan defaults. These features were chosen based on their direct relationship to creditworthiness and loan risk assessment.

### Selected Features:

- **Loan details**: amount, interest rate, grade, term
- **Financial capacity**: income, debt-to-income ratio, employment length  
- **Credit history**: FICO score, credit utilization
- **Borrower profile**: loan purpose, home ownership
- **Target**: loan status (outcome we're predicting)

These features are commonly used in credit scoring models and provide a comprehensive view of borrower risk without overwhelming the initial analysis.

In [4]:
# Select the key columns we discussed
key_columns = [
    'loan_amnt',
    'int_rate',
    'grade',
    'emp_length',
    'annual_inc',
    'dti',
    'fico_range_low',
    'revol_util',
    'purpose',
    'home_ownership',
    'loan_status'  # Target variable
]
df_subset = df[key_columns].copy()

In [5]:
df_subset.head()

,loan_amnt,int_rate,grade,emp_length,annual_inc,dti,fico_range_low,revol_util,purpose,home_ownership,loan_status
0,3600.0,13.99,C,10+ years,55000.0,5.91,675.0,29.7,debt_consolidation,MORTGAGE,Fully Paid
1,24700.0,11.99,C,10+ years,65000.0,16.06,715.0,19.2,small_business,MORTGAGE,Fully Paid
2,20000.0,10.78,B,10+ years,63000.0,10.78,695.0,56.2,home_improvement,MORTGAGE,Fully Paid
3,35000.0,14.85,C,10+ years,110000.0,17.06,785.0,11.6,debt_consolidation,MORTGAGE,Current
4,10400.0,22.45,F,3 years,104433.0,25.37,695.0,64.5,major_purchase,MORTGAGE,Fully Paid


In [6]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   loan_amnt       15000 non-null  float64
 1   int_rate        15000 non-null  float64
 2   grade           15000 non-null  object 
 3   emp_length      14105 non-null  object 
 4   annual_inc      15000 non-null  float64
 5   dti             15000 non-null  float64
 6   fico_range_low  15000 non-null  float64
 7   revol_util      14993 non-null  float64
 8   purpose         15000 non-null  object 
 9   home_ownership  15000 non-null  object 
 10  loan_status     15000 non-null  object 
dtypes: float64(6), object(5)
memory usage: 1.3+ MB


**Missing Values:**
- `emp_length`: 895 missing values (6% of data) - may need imputation or special handling
- `revol_util`: 7 missing values (0.05%) - minimal impact, can be easily handled
- All other features are complete

### Numerical Features: Summary Statistics

In [7]:
df_subset.describe()

,loan_amnt,int_rate,annual_inc,dti,fico_range_low,revol_util
count,15000.000000,15000.000000,1.500000e+04,15000.000000,15000.000000,14993.000000
mean,15071.680000,12.245799,7.826572e+04,19.442697,695.323667,51.175629
std,8735.870845,4.224309,6.208392e+04,11.949751,31.236762,24.078805
min,1000.000000,5.320000,1.770000e+03,0.000000,660.000000,0.000000
25%,8000.000000,9.170000,4.800000e+04,12.690000,670.000000,33.100000
50%,14000.000000,11.990000,6.750000e+04,18.845000,690.000000,51.000000
75%,20000.000000,14.480000,9.500000e+04,25.580000,710.000000,69.400000
max,35000.000000,28.990000,3.964280e+06,999.000000,845.000000,134.300000


### Key Insights:

**Loan Amount (`loan_amnt`):**
- Average loan: $15,072
- Range: $1,000 - $35,000
- Most loans fall between $8,000 - $20,000 (IQR)
- Fairly wide distribution suggesting diverse borrower needs

**Interest Rate (`int_rate`):**
- Average rate: 12.25%
- Range: 5.32% - 28.99%
- Wide range indicates significant risk variation among borrowers
- Median (11.99%) is close to mean, suggesting relatively symmetric distribution

**Annual Income (`annual_inc`):**
- Average: $78,266
- Median: $67,500 (lower than mean → right-skewed distribution)
- **Maximum of $3.96M is a potential outlier** - needs investigation
- Most borrowers earn between $48K - $95K

**Debt-to-Income Ratio (`dti`):**
- Average: 19.44%
- Range: 0% - 999% 
- **Maximum value of 999 is clearly an error or outlier** - requires cleaning
- Typical DTI falls between 12.7% - 25.6%

**FICO Score (`fico_range_low`):**
- Average: 695 (considered "Good" credit)
- Range: 660 - 845
- 75% of borrowers have FICO ≥ 670 (prime or near-prime credit)
- Relatively tight distribution (std = 31)

**Credit Utilization (`revol_util`):**
- Average: 51.18%
- Median: 51% (close to mean → symmetric)
- **Maximum of 134.3% indicates over-limit accounts or data issues**
- Generally, utilization >30% is considered high risk

### Data Quality Concerns:

1. **Extreme outliers detected**: 
   - Annual income: $3.96M (needs verification)
   - DTI: 999% (likely data entry error)
   - Credit utilization: >100% (over-limit scenarios or errors)



##### Investigating potential outliers

In [8]:
print(df_subset[df_subset['annual_inc'] > 500000]['annual_inc'].describe())

count    2.000000e+01
mean     9.114640e+05
std      8.311373e+05
min      5.200000e+05
25%      5.500000e+05
50%      6.000000e+05
75%      6.937500e+05
max      3.964280e+06
Name: annual_inc, dtype: float64


count    2.000000e+01     # There are 20 people with income > $500K

mean     9.114640e+05     # Average income of these 20 is $911,464

std      8.311373e+05     # Standard deviation is $831,137 (huge variation!)

min      5.200000e+05     # Lowest "high income" is $520,000

25%      5.500000e+05     # 25th percentile: $550,000

50%      6.000000e+05     # Median: $600,000

75%      6.937500e+05     # 75th percentile: $693,750

max      3.964280e+06     # Highest income: $3,964,280 (almost $4M!)

In [9]:
print(df_subset[df_subset['annual_inc'] > 3000000])

      loan_amnt  int_rate grade emp_length  annual_inc   dti  fico_range_low  \
7239     1225.0     11.99     C  10+ years   3964280.0  0.05           740.0   

      revol_util purpose home_ownership loan_status  
7239         6.7   other           RENT  Fully Paid  


For modeling purposes, I will **treat this as a data quality issue** rather than a 
legitimate outlier. The combination of:
- Tiny loan amount relative to claimed income
- Grade/FICO mismatch
- Unusual income-to-loan ratio

In [10]:
print("\nHigh DTI Outliers (>100%):")
print(df_subset[df_subset['dti'] > 100]['dti'].value_counts())


High DTI Outliers (>100%):
dti
999.00    1
137.40    1
136.97    1
Name: count, dtype: int64


### High DTI Investigation (>100%)

Only **3 records** (0.02% of dataset) have DTI above 100%:
- 999.00: 1 record (likely a placeholder for missing/unknown data)
- 137.40: 1 record (possibly legitimate - severely over-leveraged borrower)
- 136.97: 1 record (possibly legitimate - severely over-leveraged borrower)

**Interpretation:**
- **999.0** is almost certainly a **data quality issue** - this is a common placeholder value for missing or invalid data
- **137% and 136%** DTI could be real cases of borrowers with debt payments exceeding their income (financially distressed)

**Decision:**
- Replace DTI = 999 with NaN (missing value) for proper handling
- Investigate the 136-137% cases to see if they defaulted


In [11]:
print("\nCredit Utilization >100%:")
print(df_subset[df_subset['revol_util'] > 100]['revol_util'].describe())

over_100 = df_subset[df_subset['revol_util'] > 100]
print(f"\nRecords with utilization > 100%: {len(over_100)}")
print(f"Percentage of dataset: {(len(over_100)/len(df_subset)*100):.2f}%")

print("\nSample records:")
print(over_100[['revol_util', 'fico_range_low', 'grade', 'loan_status']].head())


Credit Utilization >100%:
count     74.000000
mean     102.668919
std        4.783875
min      100.100000
25%      100.600000
50%      101.200000
75%      102.675000
max      134.300000
Name: revol_util, dtype: float64

Records with utilization > 100%: 74
Percentage of dataset: 0.49%

Sample records:
     revol_util  fico_range_low grade  loan_status
23        101.5           665.0     C   Fully Paid
48        101.6           690.0     E      Current
69        102.4           680.0     E      Current
202       102.0           675.0     B   Fully Paid
249       100.6           660.0     D  Charged Off


In [ ]:
# Let's see if high utilization correlates with default
print("\nLoan Status Distribution for Utilization >100%:")
print(df_subset[df_subset['revol_util'] > 100]['loan_status'].value_counts())

print("\nCompare to overall default rate:")
print(df_subset['loan_status'].value_counts(normalize=True))

# Check the extreme case (134.3%)
print("\nExtreme utilization case (>130%):")
print(df_subset[df_subset['revol_util'] > 130][['revol_util', 'loan_amnt',
                                                   'annual_inc', 'dti',
                                                   'fico_range_low', 'grade',
                                                   'loan_status']])


Loan Status Distribution for Utilization >100%:
loan_status
Fully Paid            43
Charged Off           16
Current               14
Late (31-120 days)     1
Name: count, dtype: int64

Compare to overall default rate:
loan_status
Fully Paid            0.713800
Charged Off           0.173933
Current               0.104333
Late (31-120 days)    0.005400
In Grace Period       0.002133
Late (16-30 days)     0.000400
Name: proportion, dtype: float64

Extreme utilization case (>130%):
      revol_util  loan_amnt  annual_inc    dti  fico_range_low grade  \
9477       134.3    15000.0     55000.0  11.63           700.0     C   

     loan_status  
9477  Fully Paid  


### Outlier Investigation Summary

| Feature | Issue | Count | Decision | Rationale |
|---------|-------|-------|----------|-----------|
| Annual Income | $3.96M outlier | 1 | Remove/Cap | Inconsistent with loan amount; likely data error |
| DTI | 999% | 1 | Replace with NaN | Clear placeholder value |
| DTI | 136-137% | 2 | Keep | Legitimate severe over-leverage |
| Credit Utilization | >100% | 74 (0.49%) | **Keep** | Legitimate risk signal; 21.6% default rate vs 17.4% overall |

**Conclusion:** Credit utilization >100% is a **valuable predictor** - these borrowers are 24% more likely to default than the average borrower. This feature should be retained for modeling.